In [5]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
date = "2025-08"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [7]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [8]:
df.tail()

,CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,time,total_imports
38037,29820397,7940,ZAMBIA,DET,2025-07,29820397.0
38038,1614172,7950,ESWATINI,DET,2025-07,1614172.0
38039,6291802,7960,ZIMBABWE,DET,2025-07,6291802.0
38040,1624046,7970,MALAWI,DET,2025-07,1624046.0
38041,15845161,7990,LESOTHO,DET,2025-07,15845161.0


In [9]:
df.time.unique()[-1]

'2025-07'

In [10]:
country_list[0] = ""

In [11]:
country_list.extend(["0003", "0020"])

In [12]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '5520',
 '5830',
 '4190',
 '5330',
 '4120',
 '4759',
 '4279',
 '4419',
 '5570',
 '5490',
 '3510',
 '5590',
 '4210',
 '5600',
 '5081',
 '5170',
 '4231',
 '4700',
 '4621',
 '3010',
 '4330',
 '6021',
 '4010',
 '3370',
 '0003',
 '0020']

In [13]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)


<Response [200]>
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
5520
<Response [200]>
5830
<Response [200]>
4190
<Response [200]>
5330
<Response [200]>
4120
<Response [200]>
4759
<Response [200]>
4279
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5590
<Response [200]>
4210
<Response [200]>
5600
<Response [200]>
5081
<Response [200]>
5170
<Response [200]>
4231
<Response [200]>
4700
<Response [200]>
4621
<Response [200]>
3010
<Response [200]>
4330
<Response [200]>
6021
<Response [200]>
4010
<Response [200]>
3370
<Response [200]>
0003
<Response [200]>
0020
<Response [200]>


In [14]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL,CTY_CODE
0,USMCA (NAFTA),182150721,14,01,LIVE ANIMALS,2013-01,HS2,0020
1,USMCA (NAFTA),212532311,156,01,LIVE ANIMALS,2013-02,HS2,0020
2,USMCA (NAFTA),244602053,57,01,LIVE ANIMALS,2013-03,HS2,0020
3,USMCA (NAFTA),250009423,199,01,LIVE ANIMALS,2013-04,HS2,0020
4,USMCA (NAFTA),182211881,50,01,LIVE ANIMALS,2013-05,HS2,0020
...,...,...,...,...,...,...,...,...
14791,USMCA (NAFTA),37839253,64549,93,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...,2025-03,HS2,0020
14792,USMCA (NAFTA),25682714,74933,93,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...,2025-04,HS2,0020
14793,USMCA (NAFTA),23984117,141811,93,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...,2025-05,HS2,0020
14794,USMCA (NAFTA),30426441,300100,93,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...,2025-06,HS2,0020


In [15]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [16]:
df.index.get_level_values(0).unique()

Index(['TOTAL FOR ALL COUNTRIES', 'CHINA', 'MEXICO', 'CANADA', 'JAPAN',
       'GERMANY', 'KOREA, SOUTH', 'VIETNAM', 'TAIWAN', 'IRELAND', 'INDIA',
       'UNITED KINGDOM', 'ITALY', 'FRANCE', 'SWITZERLAND', 'MALAYSIA',
       'THAILAND', 'BRAZIL', 'SINGAPORE', 'NETHERLANDS', 'INDONESIA', 'ISRAEL',
       'SAUDI ARABIA', 'BELGIUM', 'SPAIN', 'RUSSIA', 'COLOMBIA', 'AUSTRIA',
       'AUSTRALIA', 'SWEDEN', 'CHILE', 'EUROPEAN UNION', 'USMCA (NAFTA)'],
      dtype='object', name='CTY_NAME')

In [17]:
df.loc["USMCA (NAFTA)","ALL PRODUCTS"]["2024"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2024-01-01,00,7.123938e+10,133676383.0,0.187644
2024-02-01,00,7.323562e+10,131136228.0,0.179061
2024-03-01,00,7.548168e+10,145216344.0,0.192386
2024-04-01,00,7.720258e+10,141640513.0,0.183466
2024-05-01,00,7.931487e+10,147193371.0,0.185581
2024-06-01,00,7.663457e+10,134896847.0,0.176026
2024-07-01,00,7.740682e+10,144363663.0,0.186500
2024-08-01,00,7.626432e+10,142907689.0,0.187385
2024-09-01,00,7.851141e+10,130233104.0,0.165878


In [18]:
file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [19]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [20]:
foo.tail()

I_COMMODITY       imports          duty  \
CTY_NAME I_COMMODITY_SDESC time                                                 
VIETNAM  ALL PRODUCTS      2025-03-01          00  1.483408e+10  5.067979e+08   
                           2025-04-01          00  1.474404e+10  7.122662e+08   
                           2025-05-01          00  1.579546e+10  1.111490e+09   
                           2025-06-01          00  1.634291e+10  1.516565e+09   
                           2025-07-01          00  1.776576e+10  1.640984e+09   

                                        itariff  
CTY_NAME I_COMMODITY_SDESC time                  
VIETNAM  ALL PRODUCTS      2025-03-01  3.416443  
                           2025-04-01  4.830876  
                           2025-05-01  7.036770  
                           2025-06-01  9.279648  
                           2025-07-01  9.236779

In [21]:
foo.loc["TOTAL FOR ALL COUNTRIES", "HS CODE 01, LIVE ANIMALS"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,01,221313094.0,417.0,0.000188
2013-02-01,01,246092109.0,823.0,0.000334
2013-03-01,01,261244392.0,75.0,0.000029
2013-04-01,01,271133926.0,1389.0,0.000512
2013-05-01,01,209785289.0,50.0,0.000024
...,...,...,...,...
2025-03-01,01,510175703.0,3256206.0,0.638252
2025-04-01,01,287110761.0,2465630.0,0.858773
2025-05-01,01,238290436.0,4084401.0,1.714043
